In [ ]:
# | default_exp routes.pdp

In [ ]:
# | export
import httpx
import pandas as pd
import io

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [ ]:
# | export
class PDP_NotRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        pdp_id=None,
    ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=pdp_id,
            function_name=function_name,
            status=status,
            message=message,
        )

In [ ]:
# | export
async def get_pdp_policies(
    auth: dmda.DomoAuth,
    dataset_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"http://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth, url=url, method="GET", debug_api=debug_api, session=session
    )

    if len(res.response) == 0 or not res.is_success:
        raise PDP_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name="get_pdp_policies",
            status=res.status,
            message="failed to retrieve pdp policies",
        )

    return res

## generate body for create_policy endpoint


In [ ]:
# | export
def generate_policy_parameter_simple(
    column_name: str,
    column_values_ls: list[str] = None,
    operator="EQUALS",
    ignore_case: bool = True,
):
    if not isinstance(column_values_ls, list):
        column_values_ls = [column_values_ls]

    return {
        "type": "COLUMN",
        "name": column_name,
        "values": column_values_ls,
        "operator": operator,
        "ignoreCase": ignore_case,
    }

In [ ]:
# | export
def generate_policy_body(
    policy_name: str,
    dataset_id: str,
    parameters_ls: [dict],  # generated by generate_policy_parameter_simple method
    policy_id: str = None,  # only included if updating existing policy
    user_ids: list[str] = None,
    group_ids: list[str] = None,
    virtual_user_ids: list[str] = None,
):
    if not user_ids:
        user_ids = []

    if not group_ids:
        group_ids = []

    if not virtual_user_ids:
        virtual_user_ids = []

    if not isinstance(parameters_ls, list):
        parameters_ls = [parameters_ls]


    body = {
        "name": policy_name,
        "dataSourceId": dataset_id,
        "userIds": user_ids,
        "virtualUserIds": virtual_user_ids,
        "groupIds": group_ids,
        "dataSourcePermissions": False,
        "parameters": parameters_ls,
    }

    if policy_id:
        body.update({"filterGroupId": policy_id})

    return body

#### sample implementation of generate_policy_parameter_simple and generate_policy_body


In [ ]:
from pprint import pprint

sample_parameter = generate_policy_parameter_simple(
    column_name="objectID",
    column_values_ls=["000003007"],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Policy Name",
    dataset_id ="abcdataset123",
    policy_id="123",  # including the policy_id updates that existing policy
    parameters_ls=sample_parameter,
    user_ids=['1893952720']
)

pprint(body)


{'dataSourceId': 'abcdataset123',
 'dataSourcePermissions': False,
 'filterGroupId': '123',
 'groupIds': [],
 'name': 'Test Policy Name',
 'parameters': [{'ignoreCase': True,
                 'name': 'objectID',
                 'operator': 'EQUALS',
                 'type': 'COLUMN',
                 'values': ['000003007']}],
 'userIds': ['1893952720'],
 'virtualUserIds': []}


## Policy CRUD Operations

In [ ]:
# | export
async def create_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of create_policy


In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameters = generate_policy_parameter_simple(
    column_name="objectID",
    column_values_ls=["000003007"],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Policy Name",
    dataset_id = dataset_id,
    parameters_ls= [sample_parameters],
    user_ids=[1893952720]
)

pprint(body)

# await create_policy(
#     auth = token_auth,
#     dataset_id = dataset_id,
#     body = body,
# )


{'dataSourceId': '04c1574e-c8be-4721-9846-c6ffa491144b',
 'dataSourcePermissions': False,
 'groupIds': [],
 'name': 'Test Policy Name',
 'parameters': [{'ignoreCase': True,
                 'name': 'objectID',
                 'operator': 'EQUALS',
                 'type': 'COLUMN',
                 'values': ['000003007']}],
 'userIds': [1893952720],
 'virtualUserIds': []}


ResponseGetData(status=200, response={'name': 'Test Policy Name', 'filterGroupId': 1151, 'dataSourceId': '04c1574e-c8be-4721-9846-c6ffa491144b', 'policySetId': 790, 'userIds': [1893952720], 'dataSourcePermissions': False, 'parameters': [{'name': 'objectID', 'value': '000003007', 'values': ['000003007'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': True}]}, is_success=True)

In [ ]:
# | export
async def update_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    policy_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/{policy_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of update_policy

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameters = generate_policy_parameter_simple(
    column_name="objectID",
    column_values_ls=["000003007"],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Update Policy Name",
    dataset_id=dataset_id,
    policy_id=1149,
    parameters_ls=[sample_parameters],
    user_ids=[1893952720]
)

pprint(body)

await update_policy(
    auth=token_auth,
    dataset_id=dataset_id,
    policy_id=1149,
    body=body,
)


{'dataSourceId': '04c1574e-c8be-4721-9846-c6ffa491144b',
 'dataSourcePermissions': False,
 'filterGroupId': 1149,
 'groupIds': [],
 'name': 'Test Update Policy Name',
 'parameters': [{'ignoreCase': True,
                 'name': 'objectID',
                 'operator': 'EQUALS',
                 'type': 'COLUMN',
                 'values': ['000003007']}],
 'userIds': [1893952720],
 'virtualUserIds': []}


ResponseGetData(status=200, response={'name': 'Test Update Policy Name', 'filterGroupId': 1149, 'dataSourceId': '04c1574e-c8be-4721-9846-c6ffa491144b', 'userIds': [1893952720], 'dataSourcePermissions': False, 'parameters': [{'name': 'objectID', 'value': '000003007', 'values': ['000003007'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': True}]}, is_success=True)

In [ ]:
# | export

async def toggle_pdp(auth: dmda.DomoAuth,
                     dataset_id: str,
                     is_enable: bool = True,
                     debug_api: bool = False,
                     session: httpx.AsyncClient = None
                     ):

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}"

    if debug_api:
        print(url)

    body = {"enabled": is_enable,
            "external": False  # not sure what this parameter does
            }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res


#### sample implementation of toggle_pdp

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

await toggle_pdp(
    auth=token_auth,
    dataset_id=dataset_id,
    is_enable = False,
)


ResponseGetData(status=200, response={'enabled': False, 'secured': False, 'external': False}, is_success=True)

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()